***GENERATED CODE FOR covidapp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
#%run covidappHooks.ipynb
try:
	
except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run covidappHooks.ipynb
try:
	#mlPreExecutionHook()

	clusteringautoml = Clustering.run(coronapatient,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/testdata/CoronaPatientTrain_DataCatalog.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(clusteringautoml)

except Exception as ex: 
	logging.error(ex)
